# Normalization of time series
This notebook is used as a starting point for analysing time series of radiography. 

__Disclaimer__: This procedure does not use the normalization scheme that includes black bodies. Therefore, there will be a bias in the water content calculations which predicts lower water contents than a gravimetric mesurement.

## Set up the workspace

### Prerequisites
The amglib requires that you have the package tifffile
```bash
conda install tifffile
```

### Load some packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import amglib.readers as io
import amglib.imageutils as amg

The next block is only called for development purposes. Normal notebook users don't need it.

In [35]:
import importlib
importlib.reload(amg)
importlib.reload(io)

<module 'amglib.readers' from '/Users/kaestner/git/scripts/python/amglib/readers.py'>

## Load the images
A typical time series experiment has three data sets
- __Sample images__ the image time series you want to analyze
- __Open beam images__ a set of images made without any sample in the beam
- __Dark current images__ a set of images made with closed shutters

The exmaple below reads 10 of each reference image type and averages them into a single image as 'dc' and 'ob'. The sample data is loaded as a 3D image. 
The file names are provied as a mask where the part _{:05}_ means that file names are generated with five digits and the empty places are padded by zeros. E.g. loading the file with index =42 produces the file name ```sample_00042.fits```.

_Please update data path and file masks in the cell below_

In [ ]:
datapath = '/Users/Shared/data/P20180255/00_XCalibration/'
dc=io.readImages(datapath+'dc_{:05}.fits',first=1,last=10,averageStack=True,average='mean');
ob=io.readImages(datapath+'ob_{:05}.fits',first=1,last=10,averageStack=True,average='mean');
sample=io.readImages(datapath+'xcal_{:05}.fits',first=1,last=361,stride=36)

Inspect the loaded images

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,6))
ax = ax.ravel()
amg.imshowPercentile(dc,ax[0],factor=1.96)
amg.imshowPercentile(ob,ax[1],factor=1.96)
amg.imshowPercentile(sample[0],ax[2],factor=1.96)

## Process the images

### Remove spots
Mostly there are several spots (outliers) in neutron images. These have to be removed before doing any further analysis as they may produce _inf_ and _NaN_ valued pixels in the images. The percentage sets the amount of uncorrected pixels.

In [ ]:
ob = amg.spotclean(ob,size=5,threshold=0.95) # you can probably set the threshold to 0.99 for OB and DC, 
                                             # in particular if you use average='median' while loading these images.
dc = amg.spotclean(dc,size=5,threshold=0.95)

sample = 


### Normalize
The the time series must be normalized using Beer-Lambert law 
$$I=I_0 e^{-\int_L\mu\,dx}$$
before any further analysis. In practice this looks like if you set the ```neglog=True```
$$\int_L \mu\,dx = -\log\left(\frac{D_0}{D}\cdot\frac{I_{sample}-I_{dc}}{I_{0}-I_{DC}}\right) $$
where _D_ and _D0_ are scalars computed as the average of the doseROI area.


In [ ]:
nsample = amg.normalizeImage(sample,ob,dc,neglog=True,doseROI=[200,20,400,100])

In [ ]:
N = 5 # number of images to show 

if nsample.shape[0]<N :
    N = nsample.shape[0]

fig,ax = plt.subplots(1,N,figsize=(15,6))
ax=ax.ravel()

for idx in range(N) :
    amg.imshowPercentile(sample[idx],ax[idx],factor=1.96)

## Observe relative changes 
If you want to compare the changes between different sample conditions, you have the following condition
- Initial image $I_{t=0}=I_0 e^{-L_{media}\mu_{media}}$
- Image at time _T_ $I_{t=T}=I_0 e^{-(L_{H_2O}\mu_{H_2O} + L_{media}\mu_{media})}$

Now, to see the releative change in water content you have to compute the following division:
$$ L_{H_2O}\mu_{H_2O} = -\log\left(\frac{I_{t=T}}{I_{t=0}}\right) = -\log\left(\frac{I_0 e^{-(L_{H_2O}\mu_{H_2O} + L_{media}\mu_{media})}}{I_0 e^{-L_{media}\mu_{media}}}\right) = L_{H_2O}\mu_{H_2O} + L_{media}\mu_{media} - L_{H_2O}\mu_{H_2O} $$

The last step in quantifying the water content is to divide $L_{H_2O}\mu_{H_2O}$ by $\mu_{H_2O}$, which gives you the thickness of water for each pixel. The water volume is computed using the pixel area and $L_{H_2O}$.

__Note__: As you can see, this procedure could also be done without normalizing first. It is however good practice to perform the normalization before referencing. In particular, when you intend to use scattering correction in the future.

### Provide some information for the referencing

In [ ]:
pixelSize = 100 # microns
mu_H2O    = 3.  # cm^-1 (this needs to be updated)

initialImg = nsample[0] # here, we use the first image in the time series as reference

### Reference all sample images

In [ ]:
H2O = np.zeros(nsample.shape)

for idx in range(nsample.shape[0]) :
    H2O[idx] = nsample[idx] - initialImg

### Compute local water content
The image ```H2O``` now contains the optical thickness of the sample. As the last step, we now need to rescale the values to local water content.

In [ ]:
pixelArea = pixelSize * pixelSize * 10e-8 # The scaling is needed to bring the area to cm^2

H2O = H2O/mu_H2O * pixelArea

## Your analysis follows here